In [101]:
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn import model_selection

import numpy as np
import pandas as pd

In [102]:
random.seed(42)

def get_bootstrap(data, labels, N):
    n_samples = data.shape[0]
    bootstrap = []
    
    for i in range(N):
        b_data = np.zeros(data.shape)
        b_labels = np.zeros(labels.shape)
        
        for j in range(n_samples):
            sample_index = random.randint(0, n_samples-1)
            b_data[j] = data[sample_index]
            b_labels[j] = labels[sample_index]
        bootstrap.append((b_data, b_labels))
        
    return bootstrap

In [103]:
def get_subsample(len_sample):
    # будем сохранять не сами признаки, а их индексы
    sample_indexes = [i for i in range(len_sample)]
    
    len_subsample = int(np.sqrt(len_sample))
    subsample = []
    
    random.shuffle(sample_indexes)
    for _ in range(len_subsample):
        subsample.append(sample_indexes.pop())
        
    return subsample

In [104]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [105]:
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction        

In [106]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [107]:
# Расчет качества

def quality(left_labels, right_labels, current_gini):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [108]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [109]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 1

    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    # выбор индекса из подвыборки длиной sqrt(n_features)
    subsample = get_subsample(n_features)
    
    for index in subsample:
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [110]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels):

    quality, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [111]:
def random_forest(data, labels, n_trees):
    forest = []
    bootstrap = get_bootstrap(data, labels, n_trees)
    
    for b_data, b_labels in bootstrap:
        forest.append(build_tree(b_data, b_labels))
        
    return forest

In [112]:
# Функция классификации отдельного объекта

def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [113]:
# функция формирования предсказания по выборке на одном дереве

def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [114]:
# предсказание голосованием деревьев

def tree_vote(forest, data):

    # добавим предсказания всех деревьев в список
    predictions = []
    for tree in forest:
        predictions.append(predict(data, tree))
    
    # сформируем список с предсказаниями для каждого объекта
    predictions_per_object = list(zip(*predictions))
    
    # выберем в качестве итогового предсказания для каждого объекта то,
    # за которое проголосовало большинство деревьев
    voted_predictions = []
    for obj in predictions_per_object:
        voted_predictions.append(max(set(obj), key=obj.count))
        
    return voted_predictions

In [115]:
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [116]:
TRAIN_DATASET_PATH = "C:/Users/lukfi/YandexDisk/GB/Алгоритмы анализа данных/Курсовой проект/data/train_df_prepeared.csv"
train_df = pd.read_csv(TRAIN_DATASET_PATH, index_col='Id')
train_df.head()

,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose,other_subj,efficiency,class_ver,qual_group
Id,,,,,,,,,,,,,,,,
0,35.0,2.194737,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,0,1,29.054054,0.05,1
1,52.0,2.000000,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0,1,0,21.929825,0.60,1
2,29.0,3.000000,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0,0,0,26.515152,0.17,1
3,33.0,3.000000,1050.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,1,1,15.909091,0.05,1
4,46.0,3.000000,2250.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0,0,0,30.821918,0.17,1


In [117]:
TEST_DATASET_PATH = "C:/Users/lukfi/YandexDisk/GB/Алгоритмы анализа данных/Курсовой проект/data/test_df_prepeared.csv"
test_df = pd.read_csv(TEST_DATASET_PATH, index_col='Id')
test_df.head()

,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,other_subj,efficiency,class_ver,qual_group
Id,,,,,,,,,,,,,,,
10000,32.0,2.0,2700.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0,1,30.000000,0.05,2
10001,35.0,6.0,1800.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,71.0,0,25.352113,0.28,1
10002,44.0,2.0,1200.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,1,26.666667,0.05,1
10003,44.0,4.0,2950.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,92.0,0,32.065217,0.17,2
10004,38.0,3.0,1400.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,58.0,0,24.137931,0.17,1


## Обучение модэли

In [156]:
X_TR = train_df.drop(columns = ['choose']).values
y_TR = train_df['choose'].tolist()

In [157]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_TR, y_TR, test_size=0.25)

n_trees = 50
my_forest_1 = random_forest(X_train, np.array(y_train), n_trees)

train_answers = tree_vote(my_forest_1, X_train)

test_answers = tree_vote(my_forest_1, X_test)

In [158]:
print('train',accuracy(train_answers, np.array(y_train)))
print('test',accuracy(test_answers, np.array(y_test)))

train 0.9438666666666666
test 0.89


## Применим модэль к тэстовому датасэту

In [159]:
X_TE = test_df.values

In [160]:
y_pred = tree_vote(my_forest_1, X_TE)

In [161]:
preds = pd.DataFrame()
preds['Id'] = test_df.index
preds['Choose'] = y_pred

In [162]:
preds.to_csv('C:/Users/lukfi/YandexDisk/GB/Алгоритмы анализа данных/Курсовой проект/data/preds.csv', index=False)